In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
from google import genai

c:\Users\ikath\OneDrive\GitHub\python-to-ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
generator = genai.Client(api_key=os.getenv("gemini_API"))

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1054.19it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [3]:
knowledge_base = [
    # ===== Annual Trends =====
    "Netflix generated approximately $33.7 billion in revenue in 2023, driven by steady global subscriber growth and strong engagement with original content.",
    "In 2024, Netflix revenue increased to approximately $39 billion, supported by password-sharing enforcement and expansion of ad-supported subscription plans.",
    "Early industry projections estimate Netflix revenue in 2025 may exceed $42 billion if ad-supported plans and international markets continue to grow.",
    # ===== 2023 Quarterly Trends =====
    "Netflix Q1 2023 revenue was approximately $8.16 billion, reflecting stable subscriber growth at the beginning of the year.",
    "Netflix Q2 2023 revenue was approximately $8.19 billion as international content helped sustain viewer engagement.",
    "Netflix Q3 2023 revenue increased to approximately $8.54 billion following successful original series releases.",
    "Netflix Q4 2023 revenue reached approximately $8.83 billion due to increased holiday viewing and seasonal content launches.",
    # ===== 2024 Quarterly Trends =====
    "Netflix Q1 2024 revenue increased to approximately $9.38 billion, driven by adoption of ad-supported subscription tiers.",
    "Netflix Q2 2024 revenue reached approximately $9.56 billion as password-sharing restrictions converted free users into paying subscribers.",
    "Netflix Q3 2024 revenue grew to approximately $9.82 billion, supported by strong engagement with global original content.",
    "Netflix Q4 2024 revenue peaked at approximately $10.25 billion, driven by franchise releases and high subscriber retention.",
    # ===== 2025 Early Signals & Projections =====
    "Early 2025 trends indicate Netflix revenue growth is shifting from subscriber expansion to increased monetization per user.",
    "Ad-supported subscription tiers are expected to become a major revenue driver for Netflix in 2025.",
    "Netflix’s 2025 growth strategy emphasizes content localization to expand viewership in emerging markets.",
    "Market analysts expect Netflix revenue growth in 2025 to be moderate rather than explosive due to saturation in mature streaming markets.",
    # ===== Behavioral & Strategic Insights =====
    "Netflix revenue patterns show strong seasonality, with higher performance in Q3 and Q4 driven by content release timing.",
    "Consumer willingness to accept ads in exchange for lower subscription prices has improved Netflix’s revenue resilience.",
    "Retention and engagement metrics are becoming more important than raw subscriber growth for Netflix’s long-term sustainability.",
]

In [4]:
corpus_embeddings = model.encode(knowledge_base)
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(np.array(corpus_embeddings))

In [6]:
system_prompt = """
You are a digital media consumption researcher analyzing how viewing behavior,
subscription models, and monetization strategies influence streaming platforms.
You may analyze historical trends and reason about projected future outcomes.
"""
while True:
    query = input("\n---\nenter your query regarding netflix (press q to quit):")
    if query.lower() == "q":
        break

    print(f"\nQuery: {query}")
    query_embedding = model.encode([query])

    dists, ids = index.search(np.array(query_embedding), k=3)
    context = "\n".join([knowledge_base[id] for id in ids[0]])

    print(f"\nFound context: {context}")

    response = generator.models.generate_content(
        model="gemini-2.5-flash",
        contents=f"""
{system_prompt}

Use ONLY the context below to answer the question.

Context:
{context}

Question:
{query}

Answer:
""",
    ).text
    print(f"\nResponse: {response}")


Query: What do you know about Netflix's revenue in 2023

Found context: Netflix generated approximately $33.7 billion in revenue in 2023, driven by steady global subscriber growth and strong engagement with original content.
Netflix Q1 2023 revenue was approximately $8.16 billion, reflecting stable subscriber growth at the beginning of the year.
Early industry projections estimate Netflix revenue in 2025 may exceed $42 billion if ad-supported plans and international markets continue to grow.

Response: Netflix generated approximately $33.7 billion in revenue in 2023. Additionally, Netflix's Q1 2023 revenue was approximately $8.16 billion.

Query: How did Netflix perform in 2025 as compared to 2023

Found context: Market analysts expect Netflix revenue growth in 2025 to be moderate rather than explosive due to saturation in mature streaming markets.
Early 2025 trends indicate Netflix revenue growth is shifting from subscriber expansion to increased monetization per user.
Early industry